In [3]:
import os
import io
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy
from pandas import DataFrame

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


        
def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('./spam', 'spam'))
data = data.append(dataFrameFromDirectory('./ham', 'ham'))

In [6]:
data.head()

,message,class
./spam/00294.df27a988d82cc82296e33e6d727ac47e,Get your favorite Poker action at http://www.m...,spam
./spam/00168.7422ce438a3d745e2cafb7430e5ddb0f,\n\n<html><body>\n\n<center>\n\n<table bgcolor...,spam
./spam/00326.5ec68244bb085cb140deb79563abd7b3,<$B;v6H<T(B>\n\n$B;aL>(B:Vip-mail\n\n$BFM...,spam
./spam/00253.83b95b05e275286eddcf557ea581e754,There is NO stumbling on to it! \n\n\n\nThe gr...,spam
./spam/00349.dd7982f40576ff4897c18efc813e38bf,"Dear Free Member,\n\n\n\nYou were gifted a fre...",spam


In [9]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)

classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB()

In [10]:
test = ["Free Flights to Vegas!!! Sign up below", "Hi Arnab, how are you doing?"]
test_counts = vectorizer.transform(test)
predictions = classifier.predict(test_counts)
predictions

array(['spam', 'ham'], dtype='<U4')